# Playing Around with HashTables
Here we create a hash table implementation in pure python. Mainly this is just for learning, and largely I would like to recreate a lot of the same methods used in the standard python dictionaries.  
So a few things to duplicate:
- Closed Hashing or Open Addressing for dealing with colliosn, see the following stack overflow answers:
  + [how-python-dict-stores-key-value-when-collision-occurs](https://stackoverflow.com/questions/21595048/how-python-dict-stores-key-value-when-collision-occurs)
  + [why-can-a-python-dict-have-multiple-keys-with-the-same-hash](https://stackoverflow.com/questions/9010222/why-can-a-python-dict-have-multiple-keys-with-the-same-hash)
  + [why-is-early-return-slower-than-else](https://stackoverflow.com/questions/8271139/why-is-early-return-slower-than-else)
- Compact ordered storage, see info from following links:
  + https://mail.python.org/pipermail/python-dev/2012-December/123028.html
  + [faster-more-memory-efficient-and-more](https://morepypy.blogspot.com/2015/01/faster-more-memory-efficient-and-more.html)

Apprently the Cpython implementation only uses 8 slots initially. It is then resized once it is 2/3rds full. It is my understanding that they double in size at that point.

The CPython implementation uses bitmasking instead, of the classic `hash % len(hash_table)` approach.
[This article](https://www.data-structures-in-practice.com/hash-tables/) has a great explanation of bit masking, that helped me understand what was going on.  
From what I understand, using the bit mask is faster than division on modern CPUs.

Here is the link to an example of where they are using bitmasking in the CPython dictionary implementation ([see here](https://github.com/python/cpython/blob/22415ad62555d79bd583b4a7d6a96006624a8277/Objects/dictobject.c#L867) for the code in the CPython repo).

In [1]:
%%timeit
5165093096324751164 & 31

4.99 ns ± 0.063 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


In [2]:
%%timeit
5165093096324751164 % 31

4.81 ns ± 0.0754 ns per loop (mean ± std. dev. of 7 runs, 100000000 loops each)


This this seems just as fast as the mod operator, but I think it makes sense to still use bitmasking for this example.

For the probing that happens, we want to mimic this code from the CPython source code.
```C
static Py_ssize_t
lookdict_index(PyDictKeysObject *k, Py_hash_t hash, Py_ssize_t index)
{
    size_t mask = DK_MASK(k);
    size_t perturb = (size_t)hash;
    size_t i = (size_t)hash & mask;

    for (;;) {
        Py_ssize_t ix = dictkeys_get_index(k, i);
        if (ix == index) {
            return i;
        }
        if (ix == DKIX_EMPTY) {
            return DKIX_EMPTY;
        }
        perturb >>= PERTURB_SHIFT;
        i = mask & (i*5 + perturb + 1);
    }
    Py_UNREACHABLE();
}
```

In [26]:
class Hashtable:
    def __init__(self, size=8):
        # Array size
        self.size = size
        self.sparse_key = [None] * size
        self.data = []

    def _get_index(self, hash_value):
        return hash_value & (self.size - 2)

    def add(self, key, value):
        """Add a key value pair to Hashtable"""
        if (len(self.data) + 1) >= ((2 * self.size) // 3):
            self._doublesize()

        hash_value = hash(key)
        idx = self._get_index(hash_value)
        self._add_from_index(idx=idx, hash_value=hash_value, key=key, value=value)

    def _add_from_index(self, idx, hash_value, key, value):
        if self.sparse_key[idx] is None:
            self.sparse_key[idx] = len(self.data)
            self.data.append((hash_value, key, value))
        else:
            pos = self.sparse_key[idx]
            val = self.data[pos]
            # If it's the same key/hash just replace
            if (hash_value == val[0]) and (key == val[1]):
                self.data[pos] = (hash_value, key, value)
            else:
                i = self._probe_for_index(key)
                # Treat like the probe index was the original input
                self._add_from_index(idx=i, hash_value=hash_value, key=key, value=value)

    def get(self, key):
        hash_value = hash(key)
        idx = self._get_index(hash_value)
        pos = self.sparse_key[idx]
        # If it's not none, make sure it's the
        # right value
        if pos is not None:
            val = self.data[pos]
            if (hash_value == val[0]) and (key == val[1]):
                return val[2]
            else:
                i = self._probe_for_index(key)
                pos = self.sparse_key[i]
                if pos is not None:
                    return self.data[pos][2]
        if pos is None:
            raise KeyError(f"The key {key} was not found.")

    def _probe_for_index(self, key):
        mask = self.size - 1
        hash_value = hash(key)
        perturb = hash_value
        i = self._get_index(hash_value)
        while True:
            pos = self.sparse_key[i]
            if pos is not None:
                val = self.data[pos]
                if (hash_value == val[0]) and (key == val[1]):
                    return i
            else:
                return i
            perturb >>= 5
            i = mask & ((i * 5) + perturb + 1)

    def _doublesize(self):
        """Double the size of the table"""
        self.size *= 2
        # old_data = self.data
        # self.data = []
        self.sparse_key = [None] * self.size
        # Loop through data, and update sparse key
        for slot in range(len(self.data)):
            hash_value, key, _ = self.data[slot]
            idx = self._get_index(hash_value)
            # Because the key value pairs are unique, we will
            # never be updating, only adding, so if it's not none probe.
            if self.sparse_key[idx] is not None:
                idx = self._probe_for_index(key)
            self.sparse_key[idx] = slot
            

    def __repr__(self):
        s = ""
        for _, i, j in self.data:
            s += f"{i}: {j}\n"
        return s

In [27]:
ht = Hashtable()

In [28]:
ht.add("1", 1)
ht

1: 1

In [29]:
ht.add("20", 20)
ht.add("10", 5)
ht

1: 1
20: 20
10: 5

In [30]:
ht.add("1", 40)
ht

1: 40
20: 20
10: 5

In [31]:
ht.get("1")

40

In [32]:
ht.add("30", 30)

In [33]:
ht.add("50", 50)

In [34]:
ht

1: 40
20: 20
10: 5
30: 30
50: 50

In [35]:
ht.sparse_key

[0,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 3,
 None,
 2,
 None,
 4,
 None,
 1,
 None]

In [36]:
ht.get("50")

50

In [37]:
ht.add("11", 50)

In [38]:
for i in range(30):
    ht.add(str(i), i)

In [39]:
len(ht.sparse_key)

64

In [40]:
for h, k, v in ht

1: 1
20: 20
10: 10
30: 30
50: 50
11: 11
0: 0
2: 2
3: 3
4: 4
5: 5
6: 6
7: 7
8: 8
9: 9
12: 12
13: 13
14: 14
15: 15
16: 16
17: 17
18: 18
19: 19
21: 21
22: 22
23: 23
24: 24
25: 25
26: 26
27: 27
28: 28
29: 29